## Import

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import set_config; set_config(display='diagram')

In [16]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, StackingClassifier
from xgboost import XGBClassifier


In [17]:
df = pd.read_csv('Data/train.csv')

## Data Exploration

In [4]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [6]:
df.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [5]:
df.shape

(8693, 14)

In [17]:
df.dtypes

PassengerId           object
HomePlanet            object
CryoSleep             object
Cabin                 object
Destination           object
Age                  float64
VIP                   object
RoomService          float64
FoodCourt            float64
ShoppingMall         float64
Spa                  float64
VRDeck               float64
Name                  object
Transported             bool
total_spent          float64
total_spent_high        bool
total_spent_low         bool
cabin_deck            object
cabin_num             object
cabin_side            object
AgeBinary           category
passenger_group       object
passenger_number      object
dtype: object

In [7]:
df.nunique()

PassengerId     8693
HomePlanet         3
CryoSleep          2
Cabin           6560
Destination        3
Age               80
VIP                2
RoomService     1273
FoodCourt       1507
ShoppingMall    1115
Spa             1327
VRDeck          1306
Name            8473
Transported        2
dtype: int64

In [9]:
df.isnull().sum().sort_values(ascending=False)/len(df)

CryoSleep       0.024963
ShoppingMall    0.023927
VIP             0.023352
HomePlanet      0.023122
Name            0.023007
Cabin           0.022892
VRDeck          0.021627
FoodCourt       0.021051
Spa             0.021051
Destination     0.020936
RoomService     0.020821
Age             0.020591
PassengerId     0.000000
Transported     0.000000
dtype: float64

In [12]:
df.Age.nunique()

80

## Data Preprocessing

In [ ]:
# df.duplicated().sum()

In [18]:
df['total_spent'] = df['RoomService'] + df['FoodCourt'] + df['ShoppingMall'] + df['Spa'] + df['VRDeck']
df['total_spent_log'] = np.log1p(df['total_spent'])

In [5]:
# df['total_spent_binary'] = pd.cut(x = df['total_spent'],
#                                  bins=[df['total_spent'].min()-1,
#                                        5000,
#                                        df['total_spent'].max()+1], 
#                                  labels=['low', 'high'])

In [19]:
df[['cabin_deck', 'cabin_num', 'cabin_side']] = df.Cabin.str.split("/", expand = True)

In [20]:
df['AgeBinary'] = pd.cut(x = df['Age'],
                             bins= [0,
                                    5,
                                    18,
                                    65,
                                    df['Age'].max()+1],
                             labels=['Baby','Children','Adult','Senior'])

In [7]:
# df['AgeBinary'] = pd.cut(x = df['Age'],
#                                  bins=[df['Age'].min()-1,
#                                        df['Age'].mean(),
#                                        df['Age'].max()+1], 
#                                  labels=['young', 'old'])

In [10]:
# df['Age'].max()+1

80.0

In [9]:
# df[['passenger_group', 'passenger_number']] = df.PassengerId.str.split("_", expand = True)

In [21]:
X = df.drop(columns=['PassengerId', 'Transported', 'Name', 'Cabin', 'cabin_num', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'total_spent'])
y = df.Transported
X.shape

(8693, 9)

In [9]:
X.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,total_spent_log,cabin_deck,cabin_side
0,Europa,False,TRAPPIST-1e,39.0,False,0.000000,B,P
1,Earth,False,TRAPPIST-1e,24.0,False,6.602588,F,S
2,Europa,False,TRAPPIST-1e,58.0,True,9.248021,A,S
3,Europa,False,TRAPPIST-1e,33.0,False,8.551981,A,S
4,Earth,False,TRAPPIST-1e,16.0,False,6.995766,F,S


In [22]:
X.dtypes

HomePlanet          object
CryoSleep           object
Destination         object
Age                float64
VIP                 object
total_spent_log    float64
cabin_deck          object
cabin_side          object
dtype: object

In [19]:
X.select_dtypes(include='object').nunique()

HomePlanet     3
CryoSleep      2
Destination    3
VIP            2
cabin_deck     8
cabin_side     2
dtype: int64

In [20]:
X.select_dtypes(include="float64")

,Age,total_spent_log
0,39.0,0.000000
1,24.0,6.602588
2,58.0,9.248021
3,33.0,8.551981
4,16.0,6.995766
...,...,...
8688,41.0,9.052165
8689,18.0,0.000000
8690,26.0,7.535830
8691,32.0,8.442039


In [22]:
feat_categorical = X.select_dtypes(include=['object', 'category']).columns

In [23]:
feat_numerical = X.select_dtypes(include="float64").columns

In [24]:
label_encoder = LabelEncoder()

y_encoded = label_encoder.fit_transform(y)

## Feature Correlation

## Baseline Pipe

In [11]:
preproc_numerical_baseline = make_pipeline(
    SimpleImputer(), MinMaxScaler())

preproc_categorical_baseline = make_pipeline(
    OneHotEncoder(handle_unknown="ignore"))

preproc_baseline = make_column_transformer(
    (preproc_numerical_baseline, feat_numerical),
    (preproc_categorical_baseline, feat_categorical),
    remainder="drop")

preproc_baseline

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('minmaxscaler',
                                                  MinMaxScaler())]),
                                 Index(['total_spent'], dtype='object')),
                                ('pipeline-2',
                                 Pipeline(steps=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 Index(['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'cabin_deck',
       'cabin_side', 'AgeBinary'],
      dtype='object'))])

In [20]:
pipe_baseline = make_pipeline(preproc_baseline, LogisticRegression())
pipe_baseline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  Index(['Age', 'total_spent'], dtype='object')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'cabin_deck',
       'cabin_side'],
      dtype='object'))])),
                ('logisticregressio

In [21]:
score_baseline = cross_val_score(pipe_baseline, X, y_encoded, cv=5, scoring='accuracy').mean()
score_baseline

/home/thomas/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7259879128449018

In [22]:
X_test = pd.read_csv("Data/test.csv")
X_test_ids = X_test['PassengerId']
X_test['total_spent'] = X_test['RoomService'] + X_test['FoodCourt'] + X_test['ShoppingMall'] + X_test['Spa'] + X_test['VRDeck']
X_test[['cabin_deck', 'cabin_num', 'cabin_side']] = X_test.Cabin.str.split("/", expand = True)
X_test = X_test.drop(columns=['PassengerId', 'Name', 'Cabin', 'cabin_num', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'])

pipe_baseline.fit(X,y_encoded)
y_pred_baseline = pipe_baseline.predict(X_test)
y_pred_baseline

array([1, 0, 1, ..., 1, 0, 1])

## Model Iteration

In [25]:
preproc_numerical = make_pipeline(
    SimpleImputer(strategy="median"), StandardScaler())

preproc_categorical = make_pipeline(
    SimpleImputer(strategy="most_frequent"), OneHotEncoder(handle_unknown='ignore', sparse=False))

preproc = make_column_transformer(
    (preproc_numerical, feat_numerical),
    (preproc_categorical, feat_categorical),
    remainder="drop")

preproc

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['Age', 'total_spent_log'], dtype='object')),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(sparse=False))]),
                                 Index(['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'cabin_deck',
       'cabin_side', 'AgeBinary'],
      dtype='object'))])

### Decision Tree

In [12]:
model = DecisionTreeClassifier()
pipe_decision_tree = make_pipeline(preproc, model) 
cross_val_score(pipe_decision_tree, X, y_encoded, cv=5, scoring='accuracy').mean()

0.6749100543875659

### SVC

In [19]:
model = SVC()
pipe_decision_tree = make_pipeline(preproc, model) 
cross_val_score(pipe_decision_tree, X, y_encoded, cv=5, scoring='accuracy').mean()

0.724031972133238

### Random Forest Classifier

In [18]:
model = RandomForestClassifier()
pipe_RandomForest = make_pipeline(preproc, model) 
cross_val_score(pipe_RandomForest, X, y_encoded, cv=5, scoring='accuracy').mean()

0.7045902867341057

### AdaBoost

In [19]:
model = AdaBoostClassifier()
pipe_AdaBoost = make_pipeline(preproc, model) 
cross_val_score(pipe_AdaBoost, X, y_encoded, cv=5, scoring='accuracy').mean()

0.7311644259395405

### KNN

In [20]:
model = KNeighborsClassifier(n_neighbors=3)
pipe_KNN = make_pipeline(preproc, model) 
cross_val_score(pipe_KNN, X, y_encoded, cv=5, scoring='accuracy').mean()

0.6808911646509276

### XGBoost

In [27]:
model = XGBClassifier(learning_rate=0.06, max_depth=5, n_estimators=725, reg_lambda=3, reg_alpha=4, use_label_encoder=False,
                subsample=0.8, colsample_bytree=0.8, gamma=0, min_child_weight=1, booster='gbtree', eval_metric='logloss')
                                        
pipe_XGBoost = make_pipeline(preproc, model) 
cross_val_score(pipe_XGBoost, X, y_encoded, cv=5, scoring='accuracy', n_jobs=-1).mean()

0.7440485683146604

### Voting

In [39]:
Xgboost = XGBClassifier(learning_rate=0.1, max_depth=5, n_estimators=100, reg_lambda=1, reg_alpha=1, use_label_encoder=False,
                subsample=0.8, colsample_bytree=0.8, gamma=0, min_child_weight=1, booster='gbtree', eval_metric='logloss')
Logreg = LogisticRegression()
Svc = SVC()
Adaboost = AdaBoostClassifier()


model = VotingClassifier(
    estimators = [("Xgboost", Xgboost),("Logreg", Logreg),("Svc", Svc), ("Adaboost", Adaboost)],
    weights = [1,1,1,1], # to equally weight the two models
    n_jobs=-1
)

pipe_ensemble = make_pipeline(preproc, model)

score = cross_val_score(pipe_ensemble, X, y_encoded, cv=5, scoring='accuracy', n_jobs=-1)
print(score.std())
score.mean()

0.006072144382647028


0.7396768509076617

### Stacking

In [41]:
Xgboost = XGBClassifier(learning_rate=0.1, max_depth=5, n_estimators=100, reg_lambda=1, reg_alpha=1, use_label_encoder=False,
                subsample=0.8, colsample_bytree=0.8, gamma=0, min_child_weight=1, booster='gbtree', eval_metric='logloss')
Logreg = LogisticRegression()
Svc = SVC()
Adaboost = AdaBoostClassifier()
Gnb = GaussianNB()

model = StackingClassifier(
    estimators=[("Xgboost", Xgboost),("Svc", Svc),("Adaboost", Adaboost), ("Gnb", Gnb)],
    
    final_estimator=LogisticRegression(),
    cv=5,
    n_jobs=-1
)

pipe_stacking = make_pipeline(preproc, model)
score = cross_val_score(pipe_stacking, X, y_encoded, cv=5, scoring='accuracy', n_jobs=-1)
print(score.std())
score.mean()

0.00498638530156383


0.7484192269541045

## Train and Tune the Model

In [18]:
pipe_XGBoost.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(transformers=[('pipeline-1',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(strategy='median')),
                                                    ('standardscaler',
                                                     StandardScaler())]),
                                    Index(['Age', 'total_spent'], dtype='object')),
                                   ('pipeline-2',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(strategy='most_frequent')),
                                                    ('onehotencoder',
                                                     OneHotEncoder(handle_unknown='ignore'))]),
                                    Index(['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'cabin_deck',
          'cabin_side'],
 

In [ ]:
allow_grid_searching = True

if allow_grid_searching:
    param_grid =  {'xgbclassifier__max_depth': [3, 5, 7],
                  'xgbclassifier__n_estimators': [10, 50, 100],
                   'xgbclassifier__learning_rate': [0.01, 0.1, 1],
                  }
    search_XGBoost = GridSearchCV(pipe_XGBoost, param_grid=param_grid, 
                              cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

    search_XGBoost.fit(X, y_encoded);
    svm_XGBoost_best = search_XGBoost.best_estimator_
    print(search_XGBoost.best_params_)
    search_XGBoost.best_score_

In [ ]:
allow_grid_searching = True

if allow_grid_searching:
    param_grid =  {'xgbclassifier__n_estimators': [10, 50, 100],
                   'xgbclassifier__learning_rate': [0.1, 0.3, 0.5],
                  }
    search_XGBoost = GridSearchCV(pipe_XGBoost, param_grid=param_grid, 
                              cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

    search_XGBoost.fit(X, y_encoded);
    svm_XGBoost_best = search_XGBoost.best_estimator_
    print(search_XGBoost.best_params_)
    search_XGBoost.best_score_

## Feature Selection

In [16]:
from sklearn.inspection import permutation_importance

model = pipe_XGBoost.fit(X, y_encoded)

permutation_score = permutation_importance(model, X, y_encoded, n_repeats=10)

importance_df = pd.DataFrame(np.vstack((X.columns,
                                        permutation_score.importances_mean)).T)
importance_df.columns=['feature','score decrease']

importance_df.sort_values(by="score decrease", ascending = False)

,feature,score decrease
1,CryoSleep,0.116036
5,total_spent,0.065685
6,cabin_deck,0.039342
3,Age,0.036685
7,cabin_side,0.01331
0,HomePlanet,0.012711
2,Destination,0.007765
4,VIP,0.000633


In [27]:
X = X.drop(columns=[''])

## Final Result

In [19]:
X_test = pd.read_csv("Data/test.csv")
X_test_ids = X_test['PassengerId']
X_test['total_spent'] = X_test['RoomService'] + X_test['FoodCourt'] + X_test['ShoppingMall'] + X_test['Spa'] + X_test['VRDeck']
X_test[['cabin_deck', 'cabin_num', 'cabin_side']] = X_test.Cabin.str.split("/", expand = True)
X_test = X_test.drop(columns=['PassengerId', 'Name', 'Cabin', 'cabin_num', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'])

pipe_XGBoost.fit(X,y_encoded)
y_pred_encoded = pipe_XGBoost.predict(X_test)
y_pred = label_encoder.inverse_transform(y_pred_encoded)

In [20]:
results = pd.concat([X_test_ids, pd.Series(y_pred, name="Transported")], axis=1)
results

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,False
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,False


In [22]:
results.to_csv("Data/submission_final.csv", header=True, index=False)